                        Загрузка необходимых библиотек

In [1]:
import torch

torch.cuda.is_available()

True

In [6]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import random
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

                        Функции для получения предсказания, ключевых слов, очистки текста и лемматизации

In [99]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    max1 = 0
    max2 = 0
    for i in range(len(probs[0])):
        if probs[0][i] > probs[0][max1]:
            max2 = max1
            max1 = i
    if probs[0][max2] > 0.00002:
        return f"""
№1 : {target_names[max1]}: {str(probs[0][max1])[7:13]},
№2 : {target_names[max2]}: {str(probs[0][max2])[7:13]},
"""
    return target_names[max1] + ': ' + str(probs[0][max1])[7:13]

def get_probs(text):                           #функция для получения вероятностей отношения к определенному типу договора
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to('cuda')
    outputs = model(**inputs)
    probs = outputs[0].softmax(1)
    return probs

def get_embeds(text):
    emb = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to('cpu')
    return emb['input_ids']


def cleanText(text):                            #очистка текста
    text = text.lower()
    
    text = re.sub('(-)', '', text)
    
    text = re.sub('\n', '', text)
            
    text = re.sub(r'[_\n/ _–<\*+«,\#+\№\"\-+\_+\=+{\?+\»%!+\&\}^\+\;\+\>«+"\(\)\/+\:\\+.]', r' ', text)
    
    text = re.sub(r'[abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ]', r' ', text)
    
    text = re.sub(r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)', r' ', text)
    
    text = re.sub(r'(\d+\s\d+)|(\d+)',' ', text)
    
    text = re.sub(r'\s+', ' ', text)
    
    text = re.sub(r'\uf06c\uf020', ' ', text)

    return text

def lemmatized(text):                             #лемматизация
    text = cleanText(text)
    splited = text.split(' ')
    for i in range(len(splited)):
        if len(splited[i]) == 1 or len(splited[i]) == 2:
            splited[i] = ''
        splited[i] = morph.parse(splited[i])[0].normal_form
    text = listToString(splited)
    return text
    

def get_words(model, text, size=5, how_many=5):    #получение ключевых фраз
    text = cleanText(text)
    n_gram_range = (size, size)
    count = CountVectorizer(ngram_range=n_gram_range).fit([text])
    candidates = count.get_feature_names()
    doc_embedding = model.encode([text])
    candidate_embeddings = model.encode(candidates)
    top_n = how_many
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
    return keywords

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='macro')
    return {'accuracy': acc,
            'f1'      : f1}

In [9]:
def listToString(s):
 
    # initialize an empty string
    str1 = " "
 
    # traverse in the string
    for ele in s:
        str1 += ele + " "
 
    # return string
    return str1

                        Данные, на которых будем обучать модель

In [13]:
df = pd.read_csv('last_data.csv')

labels = {'Договоры оказания услуг' : 0,
         'Договоры купли-продажи'   : 1,
         'Договоры аренды'          : 2,
         'Договоры подряда'         : 3,
         'Договоры поставки'        : 4}

target_names = ['Договоры оказания услуг',
                'Договоры купли-продажи',
                'Договоры аренды',
                'Договоры подряда',
                'Договоры поставки']

df.label = df.label.replace(labels)

cleaned = []
lemmatized = []


for text in test.text:
    text = cleanText(text)
    text2 = lemmatized(text)
    lemmatized.append(text2)
    cleaned.append(text)

test['clean'] = cleaned
test['lemmatized'] = lemmatized

                        Загрузка предобученной модели ruBert

In [14]:
model_name = "sberbank-ai/ruBert-base"
max_length = 512

tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)
vectorizer = SentenceTransformer(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))
#model.load_state_dict(torch.load('/content/doc_classifier/pytorch_model.bin'))


Downloading:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/590 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/392 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/590 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/716M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/sberbank-ai_ruBert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not in

In [30]:
X_train, X_test, y_train, y_true = train_test_split(df['lemmatized'], df['label'], random_state=42, test_size=0.2, shuffle=True)
X_train = X_train.tolist()
X_test = X_test.tolist()
y_train = y_train.tolist()
y_true = y_true.tolist()
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)

                        Приведение train и test выборки к стилю, понятному для модели

In [33]:
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsGroupsDataset(train_encodings, y_train)
valid_dataset = NewsGroupsDataset(test_encodings, y_true)

                        Параметры модели

In [34]:
training_args = TrainingArguments(
    output_dir='./results',        
    num_train_epochs=10,           
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=20,  
    warmup_steps=500,            
    weight_decay=0.01,           
    logging_dir='./logs',        
    load_best_model_at_end=True,     
    logging_steps=50,             
    save_steps=50,
    evaluation_strategy="steps", 
)

trainer = Trainer(
    model=model.to('cuda'),                 
    args=training_args,             
    train_dataset=train_dataset,  
    eval_dataset=valid_dataset,         
    compute_metrics=compute_metrics,     
)

using `logging_steps` to initialize `eval_steps` to 50
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


                        Обучение модели

In [35]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 324
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 410
  Number of trainable parameters = 178311173


Step,Training Loss,Validation Loss,Accuracy,F1
50,1.173900,1.428558,0.548780,0.399605
100,0.793100,1.069942,0.731707,0.643907
150,0.373500,0.580936,0.939024,0.938729
200,0.167700,0.318715,0.926829,0.927473
250,0.072000,0.241100,0.951220,0.950217
300,0.081900,0.332448,0.914634,0.916264
350,0.058900,0.278501,0.926829,0.927375
400,0.021000,0.278050,0.926829,0.927375


***** Running Evaluation *****
  Num examples = 82
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-50
Configuration saved in ./results/checkpoint-50/config.json
Model weights saved in ./results/checkpoint-50/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 82
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-100
Configuration saved in ./results/checkpoint-100/config.json
Model weights saved in ./results/checkpoint-100/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 82
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-150
Configuration saved in ./results/checkpoint-150/config.json
Model weights saved in ./results/checkpoint-150/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 82
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-200
Configuration saved in ./results/checkpoint-200/config.json
Model weights saved in ./results/checkpoint-200/pytorch_model.bin
***** Running E

TrainOutput(global_step=410, training_loss=0.3363094694730712, metrics={'train_runtime': 420.7461, 'train_samples_per_second': 7.701, 'train_steps_per_second': 0.974, 'total_flos': 852502781583360.0, 'train_loss': 0.3363094694730712, 'epoch': 10.0})

In [ ]:
# model_path = "doc_classifier_new"                    # Сохраняем дообученную модель
# model.save_pretrained(model_path)                    #
# tokenizer.save_pretrained(model_path)                #

Configuration saved in doc_classifier/config.json
Model weights saved in doc_classifier/pytorch_model.bin
tokenizer config file saved in doc_classifier/tokenizer_config.json
Special tokens file saved in doc_classifier/special_tokens_map.json


('doc_classifier/tokenizer_config.json',
 'doc_classifier/special_tokens_map.json',
 'doc_classifier/vocab.txt',
 'doc_classifier/added_tokens.json',
 'doc_classifier/tokenizer.json')

                        Оценка модели

In [36]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 82
  Batch size = 20


{'eval_loss': 0.24109965562820435,
 'eval_accuracy': 0.9512195121951219,
 'eval_f1': 0.9502173502173502,
 'eval_runtime': 2.6053,
 'eval_samples_per_second': 31.474,
 'eval_steps_per_second': 1.919,
 'epoch': 10.0}

                        Для теста входных текстов и функций

In [125]:
new_text = '''
Договор №______________
Дата, место заключения (населенный пункт)
_______________________________________________, именуемое в дальнейшем «Поставщик», в лице ________________________, действующего на основании ______________________, с одной стороны и
_______________________________________________, именуемое в дальнейшем «Покупатель», в лице ________________________, действующего на основании Устава, с другой стороны, именуемые в дальнейшем «Стороны», заключили настоящий договор о нижеследующем.
1. ПРЕДМЕТ ДОГОВОРА
1.1. Поставщик обязуется передать _________________________________________________(далее по тексту – «Товар») Покупателю для использования в предпринимательской деятельности или в иных целях, не связанных с личным, семейным, домашним и иным подобным использованием, а Покупатель обязуется принять и оплатить Товар в соответствии с условиями настоящего договора.
1.2. Поставка Товара осуществляется отдельными партиями в порядке и сроки определенные Спецификацией (Приложение №1) являющейся неотъемлемой частью настоящего договора.
1.3. Номенклатура (ассортимент), количество, цена поставляемого Товара и/или общая сумма каждой отдельной партии Товара указываются в Спецификации.
2. ОБЯЗАТЕЛЬСТВА СТОРОН
2.1. Покупатель обязан:
2.1.1. Принять поставленный Поставщиком Товар согласно поданным заявкам.
2.1.2. Произвести оплату Товара согласно выставленного Продавцом счета.
2.1.3. Принять Товар в момент доставки Поставщиком по количеству и ассортименту и подписать товарную накладную.
2.2. Поставщик обязан:
2.2.1. Выставлять Покупателю счета на оплату в порядке и сроки, предусмотренные настоящим договором.
2.2.2. Передать Товар в согласованные с Покупателем сроки.
2.2.3. Произвести погрузку и доставку Товара до склада Покупателя, либо произвести отгрузку Товара со склада Поставщика путем самовывоза.
2.2.4. Вместе с Товаром предоставить Покупателю документы, указанные в п. 4.3. настоящего договора.
3. СТОИМОСТЬ ТОВАРА И ПОРЯДОК РАСЧЕТОВ
3.1. Стоимость Товара указывается в выставленном счете и товарной накладной (Торг-12) на каждую партию согласно Прайс-листа (Приложение №2).
3.2. Оплата Товара производится в течение 5 (Пяти) банковских дней с момента принятия Товара и выставления счета путем безналичного перечисления денежных средств на расчетный счет Поставщика.
3.3. Датой оплаты считается день списания денежных средств с расчетного счета Покупателя.
3.4. Стороны обязуются проводить сверки взаиморасчетов по настоящему договору не реже, чем один раз в три месяца.
4. ПОРЯДОК СДАЧИ-ПРИЕМКИ ТОВАРА
4.1. Поставщик обязуется передать Покупателю Товар в сроки, согласованные Сторонами в Спецификации.
4.2. Погрузка, разгрузка и доставка Товара производится за счет Поставщика.
4.3. Вместе с Товаром Поставщик обязан передать следующие документы, оформленные надлежащим образом:
 счет-фактуру;
 товарную накладную № ТОРГ-12;
 документы, подтверждающие качество Товара (при необходимости).
5. КАЧЕСТВО ПОСТАВЛЯЕМОГО ТОВАРА
5.1. Качество Товара гарантируется Поставщиком и должно соответствовать требованиям ГОСТ, ТУ на данный вид Товара.
5.2. При обнаружении Покупателем после передачи Товара скрытых существенных недостатков (дефектов), допущенных по вине изготовителя продукции или Поставщика, Стороны составляют Акт с перечнем обнаруженных
Created with an evaluation copy of Aspose.Words. To discover the full versions of our APIs please
visit: https://products.aspose.com/words/
2
недостатков и сроками их замены. Акт о недостатках подписывается надлежащим образом уполномоченными представителями Покупателя и Поставщика (а в случае не предоставления Поставщиком своего представителя для подписания Акта в течение одного рабочего дня Покупатель вправе подписать вышеуказанный Акт в одностороннем порядке). Товар с существенными недостатками, в т.ч. оплаченный на данный момент Покупателем, подлежит замене Поставщиком за свой счет в течение 7-и дней после подписания Акта о недостатках, либо по письменной просьбе Покупателя ему может быть отпущена иная продукция на сумму возвращенной продукции или его обязательства по оплате продукции могут быть уменьшены соразмерно стоимости возвращенной продукции. Покупатель имеет право заявить Поставщику об обнаружении скрытых существенных недостатков (дефектов) Товара в течение 15 (Пятнадцати) дней с момента его приемки.
6. ОТВЕТСТВЕННОСТЬ СТОРОН
6.1. Стороны несут ответственность в рамках настоящего договора в соответствие с действующим законодательством Российской Федерации.
6.2. В случае нарушения сроков поставки Товара на Поставщика налагается штраф в размере 0,1% от цены Товара за каждый день просрочки.
6.3. Оплата штрафных санкций не освобождает Стороны от выполнения своих обязательств по настоящему договору.
6.4. Риск случайной гибели или случайного повреждения Товара переходит на Покупателя с момента подписания Сторонами товарной накладной.
6.5. В случае несоответствия Товара ГОСТам и другим обычно применяемым условиям Покупатель вправе потребовать от Поставщика уплаты штрафа в размере 10% (десяти процентов) от стоимости Товара и возмещения причиненных убытков, а так же возврат выплаченных Покупателем денежных средств за данный Товар в течение 3-х банковских дней с момента направления претензии.
6.6. В случае если Поставщик в нарушение обязанности, предусмотренной в п. 3.2., 4.3. настоящего договора, не выставил Покупателю в установленные сроки счета-фактуры на отгруженный Товар или полученные суммы предоплаты Покупатель вправе требовать от Поставщика возмещения возникших в связи с этим убытков в размере сумм НДС, которые Поставщик обязан был предъявить Покупателю в указанных счетах-фактурах.
6.7. В случае отказа Покупателю налоговыми органами в вычете сумм НДС, предъявленных Поставщиком, по причине того, что счет-фактура оформлен Поставщиком с нарушением требований главы 21 ст. 169 НК РФ, Покупатель вправе требовать от Поставщика возмещения возникших в связи с этим убытков. При этом возмещению подлежат указанные в настоящем пункте суммы НДС, а также начисленные налоговыми органами суммы пеней и штрафов, если их начисление обусловлено нарушением Поставщиком порядка составления счетов-фактур.
6.8. В обоснование требования возместить убытки, указанные в пункте 6.7. настоящего договора, Покупатель предоставляет Поставщику соответствующее решение инспекции по результатам камеральной или выездной проверки.
6.9. Проценты предусмотренные ст. 317.1 ГК РФ не начисляются Поставщиком и не оплачиваются Покупателем.
7. ФОРС-МАЖОР
7.1. Стороны освобождаются от ответственности за частичное или полное неисполнение обязательств по настоящему договору в случае, если это неисполнение явилось следствием непреодолимой силы, возникшей в результате обстоятельств чрезвычайного характера, которые Стороны не могли предвидеть или предотвратить разумными мерами.
7.2. В случаях, когда указанные обстоятельства и их последствия продолжают действовать более одного месяца, Стороны вправе согласовать иные способы исполнения настоящего договора.
8. ПОРЯДОК РАССМОТРЕНИЯ СПОРОВ
8.1. Споры и разногласия, которые могут возникнуть в ходе исполнения настоящего договора, разрешаются Сторонами путем переговоров.
8.2. В случае невозможности урегулирования споров и разногласий путем переговоров Стороны обязуются разрешать их в Арбитражном суде по месту нахождения истца.
9. КОНФИДЕНЦИАЛЬНОСТЬ
9.1. Стороны обязуются не разглашать друг о друге конфиденциальной информации, связанной с их хозяйственной деятельностью, а также не использовать во вред друг другу информацию, полученную в рамках выполнения настоящего договора.
9.2. Факт заключения и предмет настоящего договора не является конфиденциальной информацией и может использоваться Сторонами в рекламно-маркетинговых целях.
10. СРОК ДЕЙСТВИЯ И РАСТОРЖЕНИЕ ДОГОВОРА
10.1. Настоящий договор вступает в силу с момента подписания и действует до ***. Окончание срока действия договора не является основанием, прекращающим ответственность Сторон за его неисполнение.
10.2. Договор считается пролонгированным на следующий календарный год в случае, если ни одна из Сторон не
Created with an evaluation copy of Aspose.Words. To discover the full versions of our APIs please
visit: https://products.aspose.com/words/
3
заявит о его расторжении за 30 (Тридцать) календарных дней до окончания его действия.
10.3. Договор может быть расторгнут в любое время по соглашению Сторон, а также в одностороннем порядке по инициативе каждой из Сторон.
10.4. В случае расторжения настоящего договора в одностороннем порядке Сторона, принявшая решение о его расторжении, обязана письменно уведомить об этом другую Сторону. Настоящий договор считается расторгнутым с даты вручения (получения) уведомления о его расторжении, которое может быть направлено другой Стороне посредством услуг почтовой связи или вручено лично под расписку ее уполномоченному представителю.
11. ЗАКЛЮЧИТЕЛЬНЫЕ ПОЛОЖЕНИЯ
11.1. Настоящий договор составлен в двух экземплярах, каждый их которых хранится у Сторон и имеет одинаковую юридическую силу.
11.2. Все приложения, дополнения и соглашения к настоящему договору являются неотъемлемой его частью и действительны при наличии подписей лиц, наделенных правом подписания данных документов, при наличии оттисков печатей Сторон.
11.3. Любое письмо, уведомление, заявка, претензия и прочая корреспонденция (далее – Уведомление) могут быть направлены как традиционным способом, так и с использованием сети Internet. В целях исполнения Договора Стороны будут использовать адреса электронной почты указанные в п.12.2. Договора. Сообщения, полученные и отправленные по адресам электронной почты принадлежащие Сторонам либо их представителям, Стороны считают отвечающими требованиям достоверности адресата (источника) в контексте п.2 ст.434 ГК РФ. Уведомление, переданное в соответствии с установленным выше порядком, считается надлежащим и имеет юридическую и доказательственную силу. Сторона, получившая электронное сообщение, исходящее от другой стороны по Договору, обязана в течение восьми рабочих часов подтвердить получение, направив передающей стороне, сообщение о его получении (принятии).
12. АДРЕСА И БАНКОВСКИЕ РЕКВИЗИТЫ СТОРОН
12.1. В случае изменения адреса или обслуживающего банка Сторона настоящего договора обязана в течение 5 (Пяти) рабочих дней уведомить об этом в письменном виде другую Сторону.
12.2. Реквизиты сторон
Поставщик Наименование/ФИО Адрес местонахождения/регистрации Контактный номер телефона e-mail Банковские реквизиты ОГРН, ИНН, КПП Должность, ФИО, подпись Покупатель Наименование/ФИО Адрес местонахождения/регистрации Контактный номер телефона e-mail Банковские реквизиты ОГРН, ИНН, КПП Должность, ФИО, подпись
'''

In [106]:
print('Фразы: \n', get_words(vectorizer, new_text, 5, 5))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print('Предикт: \n', get_prediction(new_text))